# Krotov Optimization Setup

This notebooks shows how to set up an optimization using the `krotov` python package. Make sure to read [its documentation](https://qucontrol.github.io/krotov/v1.2.0/index.html).

In [1]:
import numpy as np

In [2]:
import krotov

In [3]:
import qutip

In [4]:
from src.ham import ryd2_hamiltonian, ket

# Defining the objectives

In [5]:
GHz = 2 * np.pi
MHz = 1e-3 * GHz
ns = 1

In [6]:
def Ω_B_left(t, args):
    E0 = 140 * MHz
    return (
        E0 * krotov.shapes.blackman(t, 0, 50) +
        E0 * krotov.shapes.blackman(t, T-50, T)
    )

def Ω_B_right(t, args):
    E0 = 55 * MHz
    return E0 * krotov.shapes.blackman(t, 0, T)

Ω_R_left = Ω_B_left
Ω_R_right = Ω_B_right

In [7]:
H = ryd2_hamiltonian(E1=(9.1 * GHz), Δ1=(1.273 * GHz), Δ2=0, u=(57.26 * MHz), Ω_B_left=Ω_B_left, Ω_B_right=Ω_B_right, Ω_R_left=Ω_R_left, Ω_R_right=Ω_R_right)

In [8]:
basis_states = [ket(label) for label in ('00', '01', '10', '11')]

In [9]:
cphase = qutip.qip.operations.cphase(np.pi)

In [10]:
cphase

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0. -1.]]

In [11]:
objectives = krotov.gate_objectives(basis_states, cphase, H)

In [12]:
objectives

[Objective[|Ψ₀(4⊗4)⟩ to |Ψ₀(4⊗4)⟩ via [H₀[4⊗4,4⊗4], [H₁[4⊗4,4⊗4], u₁(t)], [H₂[4⊗4,4⊗4], u₂(t)], [H₃[4⊗4,4⊗4], u₁(t)], [H₄[4⊗4,4⊗4], u₁(t)]]],
 Objective[|Ψ₁(4⊗4)⟩ to |Ψ₁(4⊗4)⟩ via [H₀[4⊗4,4⊗4], [H₁[4⊗4,4⊗4], u₁(t)], [H₂[4⊗4,4⊗4], u₂(t)], [H₃[4⊗4,4⊗4], u₁(t)], [H₄[4⊗4,4⊗4], u₁(t)]]],
 Objective[|Ψ₂(4⊗4)⟩ to |Ψ₂(4⊗4)⟩ via [H₀[4⊗4,4⊗4], [H₁[4⊗4,4⊗4], u₁(t)], [H₂[4⊗4,4⊗4], u₂(t)], [H₃[4⊗4,4⊗4], u₁(t)], [H₄[4⊗4,4⊗4], u₁(t)]]],
 Objective[|Ψ₃(4⊗4)⟩ to |Ψ₄(4⊗4)⟩ via [H₀[4⊗4,4⊗4], [H₁[4⊗4,4⊗4], u₁(t)], [H₂[4⊗4,4⊗4], u₂(t)], [H₃[4⊗4,4⊗4], u₁(t)], [H₄[4⊗4,4⊗4], u₁(t)]]]]

In [13]:
T = 800

In [14]:
tlist = np.linspace(0, T, 1000)

In [15]:
objectives[0].mesolve(tlist)

Result object with sesolve data.
--------------------------------
states = True
num_collapse = 0

In [16]:
def update_shape(t):
    return krotov.shapes.flattop(t, 0, T, t_rise=0.1*T)

In [17]:
pulse_options = {
    Ω_B_left: dict(lambda_a=1, update_shape=update_shape),
    Ω_B_right: dict(lambda_a=1, update_shape=update_shape),
}

## Running the optimization

This will run for a very long time ....

In [ ]:
res_opt = krotov.optimize_pulses(
    objectives,
    pulse_options,
    tlist,
    propagator=krotov.propagators.expm,
    chi_constructor=krotov.functionals.chis_sm,
    info_hook=krotov.info_hooks.print_table(J_T=krotov.functionals.J_T_sm),
)

iter.      J_T   ∑∫gₐ(t)dt          J       ΔJ_T         ΔJ  secs
0     7.25e-01    0.00e+00   7.25e-01        n/a        n/a    22
1     7.22e-01    2.34e-02   7.46e-01  -3.14e-03   2.03e-02    49 *
2     7.20e-01    1.81e-02   7.38e-01  -2.67e-03   1.55e-02    51 *
3     7.17e-01    1.67e-02   7.34e-01  -2.35e-03   1.43e-02    43 *
